# <center> Stacking

In [ ]:
%store -r X_train
%store -r X_test

%store -r y_train
%store -r y_test

%store -r X_train_sub
%store -r X_test_sub
%store -r y_train_sub
%store -r y_test_sub

In [ ]:
%store -r lgbm_hp
%store -r lgbm_cv
%store -r xgb_model_cv

In [ ]:
_da

In [ ]:
from vecstack import stacking

def create_submission(model, submission_name, send_submission=False, message="Stacking"):
    result = model.predict(X_test_sub)
    submission = pd.DataFrame({
        "ID": np.arange(result.shape[0]), 
        "item_cnt_month": result
    })
    submission.to_csv(submission_name, index=False)
   
    if send_submission:
        !kaggle competitions submit -c competitive-data-science-predict-future-sales -f {submission_name} -m {message}

In [1]:
models = [lgbm_hp, lgbm_cv, xgb_model_cv]
s1_train, s1_test = stacking(models,
                           X_train_sub, y_train_sub, X_test_sub,
                           regression=True,
                           mode='oof_pred_bag',
                           random_state=25,
                           verbose=2)

xgb_lev2 = xgb.XGBRegressor(learning_rate=0.04, 
                        n_estimators=100,
                        max_depth=3,
                        n_jobs=-1,
                        random_state=17
                       )

# Fit the 2nd level model on the output of level 1
xgb_lev2.fit(s1_train, y_train)

In [ ]:
create_submission(xgb_lev2, 'stacking_submission_hp.csv', send_submission=False)